In [ ]:
import requests

In [ ]:
import pandas as pd

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
years = list(range(2024,2020, -1))

In [ ]:
years

[2024, 2023, 2022, 2021]

In [ ]:
all_matches=[]

In [ ]:
standings_url = "https://fbref.com/en/comps/13/Ligue-1-Stats"

In [ ]:
import time
for year in years:
  data = requests.get(standings_url)
  soup = BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[0]

  links = [l.get("href") for l in standings_table.find_all('a')]
  links = [l for l in links if '/squads' in l]
  team_urls = [f"https://fbref.com{l}" for l in links]

  previous_season = soup.select("a.prev")[0].get("href")
  standings_url = f"https://fbref.com{previous_season}"

  for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")

    data = requests.get(team_url)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    try:
        team_data = matches.merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]], on="Date")
    except ValueError:
        continue

    team_data = team_data[team_data["Comp"] == "Ligue 1"]
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)
    time.sleep(1)

<ipython-input-13-4469599c75c8>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data["Season"] = year
<ipython-input-13-4469599c75c8>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data["Team"] = team_name
<ipython-input-13-4469599c75c8>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,2023-08-12,21:00,Ligue 1,Matchweek 1,Sat,Home,D,0.0,0.0,Lorient,...,Match Report,NaN,20.0,4.0,18.9,1.0,0,0,2024,Paris Saint Germain
1,2023-08-19,21:00,Ligue 1,Matchweek 2,Sat,Away,D,1.0,1.0,Toulouse,...,Match Report,NaN,15.0,2.0,15.9,0.0,1,1,2024,Paris Saint Germain
2,2023-08-26,21:00,Ligue 1,Matchweek 3,Sat,Home,W,3.0,1.0,Lens,...,Match Report,NaN,17.0,6.0,19.9,1.0,0,0,2024,Paris Saint Germain
3,2023-09-03,20:45,Ligue 1,Matchweek 4,Sun,Away,W,4.0,1.0,Lyon,...,Match Report,NaN,21.0,7.0,16.0,0.0,1,1,2024,Paris Saint Germain
4,2023-09-15,21:00,Ligue 1,Matchweek 5,Fri,Home,L,2.0,3.0,Nice,...,Match Report,NaN,12.0,3.0,20.6,1.0,0,0,2024,Paris Saint Germain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,2021-04-25,15:00,Ligue 1,Matchweek 34,Sun,Away,L,1,5,Rennes,...,Match Report,NaN,3.0,0.0,24.8,0.0,1,1,2021,Dijon
35,2021-05-02,15:00,Ligue 1,Matchweek 35,Sun,Home,L,1,5,Metz,...,Match Report,NaN,10.0,4.0,15.8,0.0,0,0,2021,Dijon
36,2021-05-09,15:00,Ligue 1,Matchweek 36,Sun,Away,L,0,3,Angers,...,Match Report,NaN,5.0,1.0,14.7,0.0,0,0,2021,Dijon
37,2021-05-16,21:00,Ligue 1,Matchweek 37,Sun,Home,L,0,4,Nantes,...,Match Report,NaN,10.0,1.0,14.7,0.0,0,0,2021,Dijon


In [ ]:
match_df.columns =[c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv("Ligue1 21-24.csv")